## UFO Sightings K-Means Clustering
### Modeling Lab

The goal of this notebook is to analyze where Mr. K should build his extraterrestrial life facilities using the K-Means algorithm. 

What we plan on accomplishing is the following:
1. [Load dataset onto Notebook instance from S3](#Step-1:-Loading-the-data-from-Amazon-S3)
2. [Cleaning, transforming, and preparing the data](#Step-2:-Cleaning,-transforming,-and-preparing-the-data)
3. [Create and train our model](#Step-3:-Create-and-train-our-model)
4. [Viewing the results](#Step-4:-Viewing-the-results)
5. [Visualize using QuickSight](https://docs.aws.amazon.com/quicksight/latest/user/create-a-data-set-s3.html)

First let's go ahead and import all the needed libraries.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

## Step 1: Loading the data from Amazon S3
Next, lets get the UFO sightings data that is stored in S3.

In [2]:
role = get_execution_role()
bucket = '<INSERT_BUCKET_NAME_HERE>'
prefix = 'ufo_dataset'
data_key = 'ufo_fullset.csv'
data_location = 's3://{}/{}/{}'.format(bucket, prefix, data_key)

df = pd.read_csv(data_location, low_memory=False)

In [3]:
df.head()

,reportedTimestamp,eventDate,eventTime,shape,duration,witnesses,weather,firstName,lastName,latitude,longitude,sighting,physicalEvidence,contact,researchOutcome
0,1982-11-29T10:01:48.297Z,1982-11-28,03:17,oval,71,1,snow,Muriel,Bartell,28.039167,-81.950000,Y,N,N,explained
1,2006-03-05T18:36:08.186Z,2006-03-05,04:56,light,75,1,partly cloudy,Floy,Heaney,33.660278,-117.998333,Y,Y,N,explained
2,2002-07-31T23:33:55.223Z,2002-07-26,13:43,oval,25,1,rain,Evelyn,Champlin,41.325278,-72.193611,Y,Y,Y,probable
3,1986-08-31T00:50:08.017Z,1986-08-27,16:12,sphere,47,1,mostly cloudy,Holden,Ward,38.254167,-85.759444,Y,N,N,explained
4,2004-09-26T08:47:39.860Z,2004-09-25,17:21,disk,59,1,rain,Abigayle,Grady,22.308085,69.600603,Y,N,N,unexplained


In [4]:
df.shape

(18000, 15)

## Step 2: Cleaning, transforming, and preparing the data
Create another DataFrame with just the latitude and longitude attributes

In [5]:
df_geo = df[['latitude', 'longitude']]

In [6]:
df_geo.head()

,latitude,longitude
0,28.039167,-81.950000
1,33.660278,-117.998333
2,41.325278,-72.193611
3,38.254167,-85.759444
4,22.308085,69.600603


In [7]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
latitude     18000 non-null float64
longitude    18000 non-null float64
dtypes: float64(2)
memory usage: 281.3 KB


In [8]:
missing_values = df_geo.isnull().values.any()
print('Are there any missing values? {}'.format(missing_values))
if(missing_values):
    df_geo[df_geo.isnull().any(axis=1)]

Are there any missing values? False


Next, let's go ahead and transform the pandas DataFrame (our dataset) into a numpy.ndarray. When we do this each row is converted to a Record object. According to the documentation, this is what the K-Means algorithm expects as training data. This is what we will use as training data for our model.

[See the documentation for input training](https://sagemaker.readthedocs.io/en/stable/kmeans.html)

In [9]:
data_train = df_geo.values.astype('float32')
data_train

array([[  28.039167,  -81.95    ],
       [  33.66028 , -117.99834 ],
       [  41.32528 ,  -72.19361 ],
       ...,
       [  37.49472 , -120.84556 ],
       [  40.771946,  -73.93056 ],
       [  64.837776, -147.71638 ]], dtype=float32)

## Step 3: Create and train our model
In this step we will import and use the built-in SageMaker K-Means algorithm. We will set the number of cluster to 10 (for our 10 sensors), specify the instance type we want to train on, and the location of where we want our model artifact to live. 

[See the documentation of hyperparameters here](https://docs.aws.amazon.com/sagemaker/latest/dg/k-means-api-config.html)

In [10]:
from sagemaker import KMeans

num_clusters = 10
output_location = 's3://' + bucket + '/model-artifacts'

kmeans = KMeans(role=role,
               instance_count=1,
               instance_type='ml.c4.xlarge',
               output_path=output_location,
               k=num_clusters)

In [11]:
job_name = 'kmeans-geo-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M%S"))
print('Here is the job name {}'.format(job_name))

Here is the job name kmeans-geo-job-20190517133512


In [14]:
%%time
kmeans.fit(kmeans.record_set(data_train), job_name=job_name)

2019-05-17 13:37:19 Starting - Starting the training job...
2019-05-17 13:37:21 Starting - Launching requested ML instances......
2019-05-17 13:38:26 Starting - Preparing the instances for training......
2019-05-17 13:39:44 Downloading - Downloading input data..

2019-05-17 13:40:12 Training - Training image download completed. Training in progress.
2019-05-17 13:40:12 Uploading - Uploading generated training model
2019-05-17 13:40:12 Completed - Training job completed
Docker entrypoint called with argument(s): train
[05/17/2019 13:40:02 INFO 140619722299200] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'_enable_profiler': u'false', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local

Billable seconds: 28
CPU times: user 695 ms, sys: 6.68 ms, total: 702 ms
Wall time: 3min 12s


## Step 4: Viewing the results
In this step we will take a look at the model artifact SageMaker created for us and stored onto S3. We have to do a few special things to see the latitude and longitude for our 10 clusters (and the center points of those clusters)

[See the documentation of deserilization here](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#td-deserialization)

At this point we need to "deserilize" the model artifact. Here we are going to open and review them in our notebook instance. We can unzip the model artifact which will contain model_algo-1. This is just a serialized Apache MXNet object. From here we can load that serialized object into a numpy.ndarray and then extract the clustered centroids from the numpy.ndarray.

After we extract the results into a DataFrame of latitudes and longitudes, we can create a CSV with that data, load it onto S3 and then visualize it with QuickSight.

In [17]:
import os
model_key = 'model-artifacts/' + job_name + '/output/model.tar.gz'

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

2304

In [18]:
!pip install mxnet

    100% |████████████████████████████████| 28.4MB 1.6MB/s eta 0:00:01  0% |▎                               | 235kB 35.0MB/s eta 0:00:01    68% |██████████████████████          | 19.6MB 45.2MB/s eta 0:00:01
    100% |████████████████████████████████| 13.8MB 5.4MB/s eta 0:00:01
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import mxnet as mx
Kmeans_model_params = mx.ndarray.load('model_algo-1')

In [20]:
cluster_centroids_kmeans = pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids_kmeans.columns=df_geo.columns
cluster_centroids_kmeans

,latitude,longitude
0,41.286369,-74.856453
1,-3.558636,115.825752
2,35.375927,-117.235794
3,48.477852,3.664200
4,36.134438,-97.897385
5,26.707329,-81.378113
6,46.405426,-120.561981
7,25.992533,-146.748108
8,38.832069,-85.299072
9,61.760826,-148.924332


Let's go ahead and upload this dataset onto S3 and view within QuickSight

In [23]:
from io import StringIO

csv_buffer = StringIO()
cluster_centroids_kmeans.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'results/ten_locations_kmeans.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5B2C5F338C4D94A2',
  'HostId': 'EBaTdqW46uapRaIWfzr0UMENSLV4vuhXsUML53S9b4QC4MP0heG2FEcRYKJqYeSum2J8ikhHdrY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'EBaTdqW46uapRaIWfzr0UMENSLV4vuhXsUML53S9b4QC4MP0heG2FEcRYKJqYeSum2J8ikhHdrY=',
   'x-amz-request-id': '5B2C5F338C4D94A2',
   'date': 'Fri, 17 May 2019 13:53:38 GMT',
   'etag': '"51e129efa7a05a163e90bd3fd0433c70"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"51e129efa7a05a163e90bd3fd0433c70"'}